Constance Delannoy

APPM 4650 | Homework 4

July 8, 2020

In [2]:
from IPython.display import HTML
import numpy as np
import pandas as pd
import math

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="The raw code is by default hidden for easier reading.To toggle on/off the raw code, click here."></form>''')

__Question 1__

We have $\int_0^1 f(x)dx = af(\frac{1}{3}) + bf(\frac{2}{3})$

Let $t = 2x-1$ so that: \begin{cases} x=0 \rightarrow t = -1 \\ = x=1 \rightarrow t-1 \\ x = (t+1)/2\\ \frac{dt}{dx} = 2\end{cases}

The problem becomes: $\int_0^1 f(x)dx = \int_{-1}^1 2d(\frac{t+1}{2})dt$.

Using the direct method, we now get:

$\int_{-1}^1 2dt = 2x \rvert_{-1}^1 = 4 \rightarrow a+b = 4$

$\int_{-1}^1 2xdt = x^2 \rvert_{-1}^1 = 0 \rightarrow \frac{a}{3} + \frac{2b}{3} = 0 \rightarrow a = -2b$

Thus: $a=8$ and $b=-4$ so that:

$\int_0^1 f(x)dx = 8f(\frac{1}{3}) - 4f(\frac{2}{3})$